In [ ]:
from google.colab import files
uploaded = files.upload()

Saving NFL.csv to NFL.csv


In [ ]:
import io
import pandas as pd
nfl_df = pd.read_csv(io.BytesIO(uploaded['NFL.csv']))
nfl_df.head()

,Year,Player,Age,School,Height,Weight,Sprint_40yd,Vertical_Jump,Bench_Press_Reps,Broad_Jump,Agility_3cone,Shuttle,Drafted..tm.rnd.yr.,BMI,Player_Type,Position_Type,Position,Drafted
0,2009,Beanie Wells\WellCh00,20.0,Ohio St.,1.8542,106.594207,4.38,85.09,25.0,325.12,NaN,NaN,Arizona Cardinals / 1st / 31st pick / 2009,31.004194,offense,backs_receivers,RB,Yes
1,2009,Will Davis\DaviWi99,22.0,Illinois,1.8796,118.387609,4.84,83.82,27.0,292.10,7.38,4.45,Arizona Cardinals / 6th / 204th pick / 2009,33.510073,defense,defensive_lineman,DE,Yes
2,2009,Herman Johnson\JohnHe23,24.0,LSU,2.0066,165.107623,5.50,NaN,21.0,NaN,NaN,NaN,Arizona Cardinals / 5th / 167th pick / 2009,41.005821,offense,offensive_lineman,OG,Yes
3,2009,Rashad Johnson\JohnRa98,23.0,Alabama,1.8034,92.079251,4.49,93.98,15.0,304.80,7.09,4.23,Arizona Cardinals / 3rd / 95th pick / 2009,28.312463,defense,defensive_back,FS,Yes
4,2009,Cody Brown\BrowCo96,22.0,Connecticut,1.8796,110.676538,4.76,92.71,26.0,304.80,7.10,4.40,Arizona Cardinals / 2nd / 63rd pick / 2009,31.327425,defense,line_backer,OLB,Yes


In [ ]:
# Import our dependencies
import matplotlib as plt
import sklearn as skl
import tensorflow as tf

# Drop columns that we won't use
bad_columns = ['Year', 'Player', 'Age', 'School', 'Drafted..tm.rnd.yr.', 'Player_Type', 'Position']
nfl_df = nfl_df.drop(bad_columns, axis=1)

# ENCODE THE POSITION_TYPE
# Create the OneHotEncoder instance
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)

# Fit the encoder and produce encoded DataFrame
nfl_df_encoded = pd.DataFrame(enc.fit_transform(nfl_df[['Position_Type', 'Drafted']].values.reshape(-2,2)))

# Rename encoded columns
nfl_df_encoded.columns = enc.get_feature_names_out(['Position_Type', 'Drafted'])

# Merge the two DataFrames together and drop the Position_Type column
nfl_df = nfl_df.merge(nfl_df_encoded,left_index=True,right_index=True).drop(["Position_Type", "Drafted"],1)

# Drop all other null values
nfl_df_encoded = nfl_df.dropna().drop("Drafted_No", axis=1)

# Create dataframe for features
X = nfl_df_encoded.drop("Drafted_Yes", axis=1)

# Create target variable
y = nfl_df_encoded.Drafted_Yes.values       

# Seperate into training and testing sets
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=5, stratify=y)

# Creating a StandardScaler instance.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
<ipython-input-4-75b24f41dd38>:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  nfl_df = nfl_df.merge(nfl_df_encoded,left_index=True,right_index=True).drop(["Position_Type", "Drafted"],1)


In [ ]:
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=31,
        step=3), activation=activation, input_dim=16))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 4)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=31,
            step=3),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [ ]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 KB 5.0 MB/s eta 0:00:00


In [ ]:
# Import the kerastuner library
import kerastuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

<ipython-input-7-c02bbece2bf0>:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [ ]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 06s]
val_accuracy: 0.7251732349395752

Best val_accuracy So Far: 0.7459584474563599
Total elapsed time: 00h 02m 29s


In [ ]:
# Get best 3 model hyperparameters
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
  print(param.values)

{'activation': 'tanh', 'first_units': 10, 'num_layers': 1, 'units_0': 31, 'units_1': 25, 'units_2': 16, 'units_3': 16, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0014'}
{'activation': 'tanh', 'first_units': 31, 'num_layers': 4, 'units_0': 25, 'units_1': 13, 'units_2': 1, 'units_3': 4, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0023'}
{'activation': 'tanh', 'first_units': 31, 'num_layers': 3, 'units_0': 13, 'units_1': 31, 'units_2': 16, 'units_3': 1, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0043'}


In [ ]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
  model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test,verbose=2)
  print(f"Loss: {model_loss}, accuracy: {model_accuracy}")

14/14 - 0s - loss: 0.5572 - accuracy: 0.7460 - 261ms/epoch - 19ms/step
Loss: 0.5571934580802917, accuracy: 0.7459584474563599
14/14 - 1s - loss: 0.5599 - accuracy: 0.7436 - 586ms/epoch - 42ms/step
Loss: 0.5598651766777039, accuracy: 0.7436489462852478
14/14 - 0s - loss: 0.5630 - accuracy: 0.7436 - 447ms/epoch - 32ms/step
Loss: 0.5630006790161133, accuracy: 0.7436489462852478


In [ ]:
nn_model1 = tf.keras.models.Sequential()

nn_model1.add(tf.keras.layers.Dense(units=10, activation='tanh', input_dim=16))
nn_model1.add(tf.keras.layers.Dense(units=31, activation='tanh'))
nn_model1.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

nn_model1.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

In [ ]:
# Fit the model to the training data
fit_model = nn_model1.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
41/41 [==============================] - 2s 4ms/step - loss: 0.6910 - accuracy: 0.5424
Epoch 2/50
41/41 [==============================] - 0s 2ms/step - loss: 0.6307 - accuracy: 0.6795
Epoch 3/50
41/41 [==============================] - 0s 2ms/step - loss: 0.6030 - accuracy: 0.6941
Epoch 4/50
41/41 [==============================] - 0s 2ms/step - loss: 0.5906 - accuracy: 0.6895
Epoch 5/50
41/41 [==============================] - 0s 2ms/step - loss: 0.5829 - accuracy: 0.6980
Epoch 6/50
41/41 [==============================] - 0s 2ms/step - loss: 0.5794 - accuracy: 0.6965
Epoch 7/50
41/41 [==============================] - 0s 1ms/step - loss: 0.5760 - accuracy: 0.7026
Epoch 8/50
41/41 [==============================] - 0s 2ms/step - loss: 0.5751 - accuracy: 0.7011
Epoch 9/50
41/41 [==============================] - 0s 2ms/step - loss: 0.5719 - accuracy: 0.7072
Epoch 10/50
41/41 [==============================] - 0s 2ms/step - loss: 0.5709 - accuracy: 0.6995
Epoch 11/50
41/41 [

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

14/14 - 0s - loss: 0.5648 - accuracy: 0.7367 - 374ms/epoch - 27ms/step
Loss: 0.5648136138916016, Accuracy: 0.7367205619812012


In [ ]:
nn_model1_1 = tf.keras.models.Sequential()

nn_model1_1.add(tf.keras.layers.Dense(units=10, activation='tanh', input_dim=16))
nn_model1_1.add(tf.keras.layers.Dense(units=31, activation='tanh'))
nn_model1_1.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

nn_model1_1.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

In [ ]:
# Fit the model to the training data
fit_model = nn_model1_1.fit(X_train_scaled, y_train, epochs=20)

Epoch 1/20
41/41 [==============================] - 1s 1ms/step - loss: 0.6620 - accuracy: 0.5986
Epoch 2/20
41/41 [==============================] - 0s 2ms/step - loss: 0.6047 - accuracy: 0.6810
Epoch 3/20
41/41 [==============================] - 0s 1ms/step - loss: 0.5827 - accuracy: 0.7011
Epoch 4/20
41/41 [==============================] - 0s 2ms/step - loss: 0.5742 - accuracy: 0.7049
Epoch 5/20
41/41 [==============================] - 0s 1ms/step - loss: 0.5707 - accuracy: 0.7096
Epoch 6/20
41/41 [==============================] - 0s 1ms/step - loss: 0.5674 - accuracy: 0.7111
Epoch 7/20
41/41 [==============================] - 0s 2ms/step - loss: 0.5638 - accuracy: 0.7173
Epoch 8/20
41/41 [==============================] - 0s 1ms/step - loss: 0.5617 - accuracy: 0.7180
Epoch 9/20
41/41 [==============================] - 0s 2ms/step - loss: 0.5584 - accuracy: 0.7234
Epoch 10/20
41/41 [==============================] - 0s 2ms/step - loss: 0.5570 - accuracy: 0.7188
Epoch 11/20
41/41 [

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model1_1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

14/14 - 0s - loss: 0.5573 - accuracy: 0.7390 - 130ms/epoch - 9ms/step
Loss: 0.557344913482666, Accuracy: 0.7390300035476685
